In [ ]:
f_func <- "./functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))

f_data_loading <- "./data_loading.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_data_loading, '--to=script', '--stdout'), stdout = TRUE)))

In [ ]:
combined_data <- rbind(
    plasma_sizeSelection_meta %>% filter(new_Filter == "35-100"),
    BAL_sizeSelection_meta %>% filter(new_Filter == "35-100")
) 

sel_combined_data <- combined_data %>% 
    filter(taxName == "Aspergillus fumigatus")

Asp_species_count <- sel_combined_data %>% 
    select(c("taxName", "taxID_number_reads", "database", "CT","sample_id")) %>% 
    rename("db" = database) %>% 
    rename("threshold" = CT) %>% 
    mutate(threshold = gsub(threshold, pattern = "conf", replacement = "")) %>% 
    rename("count" = taxID_number_reads, ) %>% 
    rename("sample" = sample_id) 

total_read_count <- data.frame("EPRSc2" = sel_combined_data$QC_number_reads, row.names = sel_combined_data$sample_id) %>% t()

In [ ]:
Asp_species_count %>% head()
total_read_count %>% head()

In [ ]:
Bctrls <- c('H01Bctrl','H05Bctrl','H06Bctrl','H07Bctrl','H08Bctrl','H10Bctrl','H11Bctrl','H22Bctrl','H24Bctrl')
Pctrls <- c('H31Pctrl','H32Pctrl','H33Pctrl','H34Pctrl','H35Pctrl','H36Pctrl','H37Pctrl','H38Pctrl','H39Pctrl')

ctrl_list <- list(Bctrls, Pctrls)
names(ctrl_list) <- c("Bctrls", "Pctrls")

In [ ]:
dbs_sel_min <- c('EPRSc2')
lst_fisher_species <- list()
for (fav_db in dbs_sel_min){
    if (fav_db == "EPRSc2"){print(fav_db); tmp_species_threshold = "conf0.4"}
    
    df_fisher_species <- Asp_species_count %>% 
        filter(db == fav_db) %>% 
        filter(threshold == gsub(tmp_species_threshold, pattern = "conf", replacement = "")) %>% 
        filter(grepl(sample, pattern = "asp$") | 
               grepl(sample, pattern = "ctrl$") | 
               grepl(sample, pattern = "ctrl2$") | 
               grepl(sample, pattern = "ctrl3$") | 
               grepl(sample, pattern = "VAL$")
              ) %>% 
        mutate(control_type = ifelse(grepl(sample, pattern = "Bctrl") | grepl(sample, pattern = "Basp"), 
                                     yes = "Bctrls", no = 
                              ifelse(grepl(sample, pattern = "Pctrl") | grepl(sample, pattern = "Pasp"), 
                                     yes = "Pctrls", no = NA))) %>% 
        mutate(origin = ifelse(control_type == "Bctrls", yes = "BAL ss-cfDNA", no = 
                        ifelse(control_type == "Pctrls", yes = "Plasma ss-cfDNA", no = NA))) %>% 
        mutate(fisher.t.ext = NA) %>%  mutate(fisher.t.int = NA) 

    for (i in 1:nrow(df_fisher_species)){
        db_tmp <- df_fisher_species$db[i]
        ctrl <- df_fisher_species$control_type[i]
        tn <- df_fisher_species$taxName[i]
        smpl <- df_fisher_species$sample[i]

        tmp_ext <- c()
        for (ext_c in ctrl_list[[ctrl]]){
            if(ext_c == c(df_fisher_species$sample[i])){next}
            ext_ctrl_species_count <- Asp_species_count %>% 
                filter(sample == ext_c) %>% 
                filter(db == db_tmp) %>% 
                filter(taxName == tn) %>% 
                filter(threshold == gsub(tmp_species_threshold, pattern = "conf", replacement = "")) %>% 
                pull("count")

            ft_ext <- as.numeric(fisher.test(
                            rbind(c(df_fisher_species$count[i],
                                    (total_read_count[db_tmp, smpl]-df_fisher_species$count[i])),
                                  c(ext_ctrl_species_count, 
                                    (total_read_count[db_tmp, ext_c])-ext_ctrl_species_count)), 
                            alternative="greater")$p.value)
            tmp_ext <- c(tmp_ext, ft_ext)
            rm(ft_ext)
        }
        df_fisher_species$fisher.t.ext[i] <- mean(tmp_ext, na.rm = TRUE)

        if(ctrl == "Pctrls" & df_fisher_species$sample[i] %nin% c('A05Pctrl2','A05Pctrl3')){
            int_c <- gsub(df_fisher_species$sample[i], pattern = "asp", replacement = "ctrl")
            if(int_c %in% Asp_species_count$sample & int_c %in% colnames(total_read_count)){
                int_ctrl_species_count <- Asp_species_count %>% 
                    filter(sample == ext_c) %>% 
                    filter(db == db_tmp) %>% 
                    filter(taxName == tn) %>% 
                    filter(threshold == gsub(tmp_species_threshold, pattern = "conf", replacement = "")) %>% 
                    pull("count")

                df_fisher_species$fisher.t.int[i] <- as.numeric(fisher.test(
                                rbind(c(df_fisher_species$count[i],
                                        (total_read_count[db_tmp, smpl]-df_fisher_species$count[i])),
                                      c(int_ctrl_species_count, 
                                        (total_read_count[db_tmp, int_c])-int_ctrl_species_count)), 
                                alternative="greater")$p.value)
            }
        }
    }
    lst_fisher_species[[fav_db]] <- df_fisher_species
}

In [ ]:
lst_fisher_species$EPRSc2 %>% filter(fisher.t.ext < 0.001) %>% 
    select(c("sample", "origin", "count", "fisher.t.ext"))

lst_fisher_species$EPRSc2 %>% #filter(fisher.t.ext < 0.001) %>% 
    filter(sample == "A03Pasp") %>% 
    select(c("sample", "origin", "count", "fisher.t.ext"))